In [25]:
import re

import pandas as pd
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv("../data/temp_data/all_data.csv")

# Переделаю датасет до аугментации, чтобы убрать из названий нумерацию (том, эпизод и т.п.)

In [39]:
data.head()

,title,text,title_len
0,время - октябрь...,­­… сказки осенних вечеров ... монохромные во...,3
1,переписка,"здравствуйте, вы установили мессенджер max, по...",1
2,айвенго (спектакль) часть 3-я. ричард – львино...,радиоспектакль по роману.англия времен царство...,8
3,калевала,где можно найти наиболее полный литературный о...,1
4,янки из коннектикута при дворе короля артура,"новый, адаптированный перевод алексея козлова ...",7


In [40]:
def remove_complex_volume_numbers(title):
    """
    Удаляет сложные формы нумерации: (139 эпизод 2 том 2.)
    """
    if pd.isna(title):
        return title

    title_str = str(title)

    patterns = [
        r'\(?\s*\d+\s+(?:эпизод|серия|глава|часть)\s+\d+\s+(?:том|книга|т\.)\s*\d*\s*\.?\)?',
        r'\(?\s*\d+\s+(?:том|книга|т\.)\s+\d+\s+(?:эпизод|серия|глава|часть)\s*\d*\s*\.?\)?',

        r'\b\d+\s+\d+\s+(?:том|часть|книга|эпизод|серия)\b',
        r'\b(?:том|часть|книга|эпизод|серия)\s+\d+\s+\d+\b',

        r'\b\d+[-\.,]\s*\d+\s+(?:том|часть|книга)',
        r'\b(?:том|часть|книга)\s+\d+[-\.,]\s*\d+\b',
    ]

    cleaned_title = title_str
    for pattern in patterns:
        cleaned_title = re.sub(pattern, '', cleaned_title, flags=re.IGNORECASE)

    return cleaned_title

def clean_title_completely(title):
    """
    Полная очистка названия от всех видов нумерации
    """
    if pd.isna(title):
        return title

    cleaned = remove_complex_volume_numbers(title)

    basic_patterns = [
        r'\s*(?:том|часть|книга|т\.|vol\.?|эпизод|серия|глава|выпуск)\s*[ivxlcdm0-9]+',
        r'\s*[ivxlcdm0-9]+\s*(?:том|часть|книга|т\.|vol\.?|эпизод|серия|глава|выпуск)',
        r'\s*\d+[-\.,]?\s*(?:том|часть|книга)',
        r'\s*(?:том|часть|книга)[-\.,]?\s*\d+',
    ]

    for pattern in basic_patterns:
        cleaned = re.sub(pattern, '', cleaned, flags=re.IGNORECASE)

    cleaned = re.sub(r'\([^)]*\d+[^)]*\)', '', cleaned)
    cleaned = re.sub(r'\s+\d+\s*\.?$', '', cleaned)
    cleaned = re.sub(r'^\d+\s+', '', cleaned)

    cleaned = re.sub(r'\s+', ' ', cleaned)
    cleaned = re.sub(r'^\s+|\s+$', '', cleaned)
    cleaned = re.sub(r'[\(\)\[\]]', '', cleaned)
    cleaned = re.sub(r'[,\-\.:;]+$', '', cleaned)

    return cleaned


In [41]:
data["cleaned_title"] = data.title.apply(clean_title_completely)

In [42]:
changed_count = (data['title'] != data['cleaned_title']).sum()
print(f"Изменено названий: {changed_count} из {len(data)}")

Изменено названий: 5801 из 31122


In [43]:
data.head()

,title,text,title_len,cleaned_title
0,время - октябрь...,­­… сказки осенних вечеров ... монохромные во...,3,время - октябрь
1,переписка,"здравствуйте, вы установили мессенджер max, по...",1,переписка
2,айвенго (спектакль) часть 3-я. ричард – львино...,радиоспектакль по роману.англия времен царство...,8,айвенго спектакль-я. ричард – львиное сердце
3,калевала,где можно найти наиболее полный литературный о...,1,калевала
4,янки из коннектикута при дворе короля артура,"новый, адаптированный перевод алексея козлова ...",7,янки из коннектикута при дворе короля артура


In [44]:
data.title_len = data.cleaned_title.apply(lambda x: len(str(x).split()))

In [45]:
data.head()

,title,text,title_len,cleaned_title
0,время - октябрь...,­­… сказки осенних вечеров ... монохромные во...,3,время - октябрь
1,переписка,"здравствуйте, вы установили мессенджер max, по...",1,переписка
2,айвенго (спектакль) часть 3-я. ричард – львино...,радиоспектакль по роману.англия времен царство...,8,айвенго спектакль-я. ричард – львиное сердце
3,калевала,где можно найти наиболее полный литературный о...,1,калевала
4,янки из коннектикута при дворе короля артура,"новый, адаптированный перевод алексея козлова ...",7,янки из коннектикута при дворе короля артура


In [47]:
data.describe(include="O")

,title,text,cleaned_title
count,31122,31101,31122
unique,28839,30640,27913
top,латырь-камень бел-горюч... сказка?,; ; «энцикловикипедия»- -имена. посвящается ми...,латырь-камень бел-горюч... сказка?
freq,38,38,38


In [49]:
data = data[~data.text.duplicated(keep=False)].reset_index(drop=True)

In [50]:
data.describe(include="O")

,title,text,cleaned_title
count,30588,30588,30588
unique,28730,30588,27806
top,***,­­… сказки осенних вечеров ... монохромные во...,
freq,20,1,28


In [53]:
data = data[data.cleaned_title != ""].reset_index(drop=True)
data.describe(include="O")

,title,text,cleaned_title
count,30560,30560,30560
unique,28705,30560,27805
top,***,­­… сказки осенних вечеров ... монохромные во...,классика зарубежного рассказа №
freq,20,1,25


In [54]:
data

,title,text,title_len,cleaned_title
0,время - октябрь...,­­… сказки осенних вечеров ... монохромные во...,3,время - октябрь
1,переписка,"здравствуйте, вы установили мессенджер max, по...",1,переписка
2,айвенго (спектакль) часть 3-я. ричард – львино...,радиоспектакль по роману.англия времен царство...,8,айвенго спектакль-я. ричард – львиное сердце
3,калевала,где можно найти наиболее полный литературный о...,1,калевала
4,янки из коннектикута при дворе короля артура,"новый, адаптированный перевод алексея козлова ...",7,янки из коннектикута при дворе короля артура
...,...,...,...,...
30555,что бывало,борис житков увлекался астрономией и фотографи...,2,что бывало
30556,сон макара,"«этот сон видел бедный макар, который загнал с...",2,сон макара
30557,финал. золотое солнце правосудия,раннее утро предвещало жаркий день. солнечные ...,4,финал. золотое солнце правосудия
30558,святочные истории,святки – удивительный период времени: один год...,2,святочные истории


In [55]:
data.drop(columns=["title", "title_len"], inplace=True)

In [56]:
data

,text,cleaned_title
0,­­… сказки осенних вечеров ... монохромные во...,время - октябрь
1,"здравствуйте, вы установили мессенджер max, по...",переписка
2,радиоспектакль по роману.англия времен царство...,айвенго спектакль-я. ричард – львиное сердце
3,где можно найти наиболее полный литературный о...,калевала
4,"новый, адаптированный перевод алексея козлова ...",янки из коннектикута при дворе короля артура
...,...,...
30555,борис житков увлекался астрономией и фотографи...,что бывало
30556,"«этот сон видел бедный макар, который загнал с...",сон макара
30557,раннее утро предвещало жаркий день. солнечные ...,финал. золотое солнце правосудия
30558,святки – удивительный период времени: один год...,святочные истории


In [57]:
data.rename(columns={"cleaned_title": "title"}, inplace=True)

In [59]:
data.head()

,text,title
0,­­… сказки осенних вечеров ... монохромные во...,время - октябрь
1,"здравствуйте, вы установили мессенджер max, по...",переписка
2,радиоспектакль по роману.англия времен царство...,айвенго спектакль-я. ричард – львиное сердце
3,где можно найти наиболее полный литературный о...,калевала
4,"новый, адаптированный перевод алексея козлова ...",янки из коннектикута при дворе короля артура


In [60]:
data.to_csv("../data/temp_data/all_data.csv", index=False)